<p align="center">
  <img src="Graphics/Episode V.png" />
</p>

## (0) Introduction to Temporal & Numeric Planning

### (0.1) Learning Outcomes

In this tutorial, we will cover:
* The basics of temporal planning and numeric planning
* How can we solve temporal and numeric planning problems in Python using the Unified Planning Framework (and PDDL 2.1)?

<p align="center">
  <img src="Graphics/Sched.png" width=700/>
</p>

## (1) PDDL 2.1: Adding Support for Temporal Planning & Numeric Planning

Up until now, we've been working with classical planning problems where all actions are assumed to be *instantaneous*, and such problems were sufficiently modeled by PDDL 1.2 (released in 1998). This is rarely ever a good model of the real world though, since tasks can take time to complete and tasks can even be concurrent (occuring at the same moment in time). This led to the development of a big update to PDDL in 2003 known as [PDDL 2.1](https://arxiv.org/pdf/1106.4561.pdf), which uses the same basic syntax of PDDL 1.2 but with added support for *durative actions* (actions which take time to complete) as well as *numeric fluents* (variables with integer or real number values which can change over time).

From now on, we will be able to use PDDL 2.1 to model the following types of planning problems:
* **Temporal Planning:** Addresses the problem of generating a sequence of actions that transform the environment from some initial state to a desired goal state, while taking into account time constraints and the durations of actions
* **Numeric Planning:** Addresses problems where numbers (integer/real) must be used in the planning model, i.e. actions have numeric preconditions and effects, and goals can be numeric

### (1.1) PDDL 2.1 Syntax

#### (1.1.1) Numeric Fluents

Let's take a look at the new features of PDDL 2.1 which allow us to represent and treat temporal and numeric planning problems. First, we'll have to add some new requirements to our domain file, called `:durative-actions` and `:fluents`. We can then define our types as usual, and we'll see a new feature when we begin to define our numeric fluents, which we do in a group called `(:functions)`. An example of numeric fluent definition is shown below: 
```
(:functions
        (battery-amount ?r - rover)
        (sample-amount ?r - rover)
        (recharge-rate ?r - rover)
        (battery-capacity)
        (sample-capacity)
        (distance-travelled)
)
```
All of these fluents store some numeric value (can be integer or real), and can even be tied to some object (for example, the battery amount of a rover, which is specific to that rover). These fluents can be altered in both regular (instantaneous) actions and durative actions. After we define the numeric fluents, we can define our predicates (which are boolean) as usual.

How can we change the values of numeric fluents? PDDL 2.1 offers support for the following numeric operators:
* **Addition:** Adds some value to a fluent, i.e. `(+ (sample-capacity) (battery-capacity))`
* **Subtraction:** Substracts some value from a fluent, i.e. `(- (sample-capacity) (battery-capacity))`
* **Multiplication:** Multiplies a fluent by some value, i.e. `(* (sample-capacity) (battery-capacity))`
* **Division:** Divides a fluent by some value, i.e. `(/ (sample-capacity) (battery-capacity))`

We can describe the same numeric effects using keywords:
* **Increase:** Increases a fluent by some value, i.e. `(increase (battery-level ?r) 10)`; it is possible to use another numeric variable as the increase value (e.g. `(increase (battery-level ?r) (charge-available - ?solarpanel))`)
* **Decrease:** Decreases a fluent by some value, i.e. `(decrease (battery-level ?r) 10)`, or `(decrease (battery-level ?r) (power-needed-for-work - ?task))`
* **Scale Up:** Multiplies the value of the numeric variable by the given scale factor, i.e. `(scale-up (battery-level ?r) 2)`, or `(scale-up (battery-level ?r) (charge-rate ?r))`
* **Scale Down:** Divides the value of the numeric variable by the given scale factor, i.e. `(scale-down (battery-level ?r) 2)`, or `(scale-down (battery-level ?r) (consumption-rate ?r))`
* **Assign:** Assigns a value to a fluent, i.e. `(assign (battery-level ?r) 10)`, or `(assign (battery-level ?r) (max-charge ?r))` (this doesn't have a counterpart above)

We can also use equalities and inequalities to define action conditions and effects, initial values, and goals involving numeric fluents. For example, if we look ahead to our problem file, we will define the initial values of the numeric fluents in the `(:init)` group along with the predicates, using the following kind of notation:
```
    (:init
        (= (battery-amount r1) 100)
        (= (recharge-rate r1) 2.5)
        ...
    )
```
We just use an equals sign (=) to assign the values of the fluents. The goal state for numeric fluents can be defined similarly.

#### (1.1.2) Durative Actions

Now back to the domain file, once we've defined our fluents and predicates we can begin to define our durative actions, which have a duration that can either be fixed (and equal to some number) or bounded in some interval (to do that, we'd need to add the requirement `:duration-inequalities`). To define a durative action, we start by declaring its name and parameters as follows:
```
(:durative-action move
        :parameters (?r - rover ?from ?to - location)
```
Nothing unusual so far (except for declaring the action using the `:durative-action` command). Next, we add a new feature which records the duration of the action:
```
        :duration (= ?duration 5)
```
Here, the duration of the action is fixed and it takes 5 units of time to complete. We could also use an inequality symbol here in order to indicate that the duration is not fixed, only bounded by some value(s). Next, we have to state the conditions (*not* preconditions like before) that must hold at the beginning, middle, and/or end of the action:
```
        :condition (and (at start (at ?rover ?from))
	                (over all (can-move ?from ?to))
	                (at start (> (battery-amount ?rover) 8)) 
                    )
```
Why don't we call it a precondition? Well, a condition for a durative action can be specified at *any* given moment during the action's duration, not just at the beginning (i.e. a condition must hold at the end of the action isn't technically a precondition). Looking at our condition definition, we notice the following new features:
* `at start`: indicates that this condition must hold at the time when the action begins
* `over all`: indicates that this condition must hold for the whole duration of the action
* `at start (> ({fluent}) {value}))`: the inequality here tells us that the fluent in question must be greater than the specified value at the start of the action, in order to actually do the action

Finally, we can describe the effects of our durative action (which will also be temporal). Here's an example:
```
        :effect (and (at end (at ?rover ?to)) 
	             (at end (been-at ?rover ?to))
	             (at start (not (at ?rover ?from))) 
	             (at start (decrease (battery-amount ?rover) 8))
                     (at end (increase (distance-travelled) 5))
                )
```
Some more new features to notice:
* `at end`: indicates that this effect will occur once the action ends
* `at start`: indicates that this effect will occur immediately after the action begins
* `at start (decrease ({fluent}) {value}))`: indicates that the numeric fluent in question will decrease by the specified value immediately after the action begins
* `at end (increase ({fluent}) {value}))`: indicates that the numeric fluent in question will increase by the specified value once the action ends

#### (1.1.3) Continuous Effects

Another new feature in temporal/numeric planning problems is something called a *continuous effect*, which is another way of defining effects on numeric variables where their values can now change throughout the application of a durative action. Put simply, it allows us to change variables continously over the duration of the action. For example, we can define such effects as follows:
```
        (increase (fuel ?tank) #t)
        (decrease (battery ?battery) (* 5 #t))
```
Here, the fuel level in the tank increases directly with time (i.e. for every unit of time that passes during the action, the value of `fuel` is increased by 1). Similarly, the battery level here will decrease proportionally with time (i.e. for every unit of time that passes during the action, the value of `battery` is decreased by 5).

We can see these kind of effects in real world fuel and battery problems. As we drive for longer periods of time, we consume more fuel, and this can be modelled using a continous effect. Furthermore, continuous effects allow planning models to consider the application of an action prior to the termination of a durative action.

#### (1.1.4) Metrics

Now we've finished defining our domain file (we can of course add other actions which are instantaneous alongside the durative ones), and we will define our problem file almost identically to how we usually might do it (small changes to the initial and goal state definitions). One last new feature, though, will be something called a *metric*. When dealing with numeric planning problems, we might want to ask our planner to choose the solution which maximizes or minimizes certain values (fluents). The planner will then try to optimize the search according to the *metric* we requested. We can declare metrics as follows:
```
(:metric minimize (<numeric_operation>)) ; to minimize the specified fluent
(:metric maximize (<numeric_operation>)) ; to maximize the specified fluent
```
Note that metrics do *not* replace goals, they are just used to declare what optimality means for a solution to our problem.

### (1.2) Integrating PDDL 2.1 with Python using the Unified Planning Framework (UPF)

For those of you who would like to learn more about the new features of PDDL 2.1, feel free to check out the [following page](https://planning.wiki/ref/pddl21) for more info. The [original paper](https://arxiv.org/pdf/1106.4561.pdf) for PDDL 2.1, written by Maria Fox and Derek Long, is also a great resource.

Now we'd like to actually apply PDDL 2.1 to real planning problems, and to do so we will once again make our lives easier by using the Unified Planning Framework (UPF). Luckily for us, it supports PDDL 2.1 and temporal/numeric planning in general. Let's see how we can use the UPF to represent and solve such problems with a few examples.

## (2) Ex: Match Cellar

<p align="center">
  <img src="Graphics/Match.jpg" width=1000/>
</p>

### (2.1) Problem Statement

The match cellar problem is a classic example (and essentially the "Hello World") for temporal planning problems, and you will have seen it already in [this week's lecture](https://docs.google.com/presentation/d/1N5gc__kmY3TE9cHqFg2zM8yT1zxehTc710nCRk6otxM/edit#slide=id.g221c7f3818_1_0). In this problem, we need to change some fuses in our fusebox, located in our cellar which has no natural or electric lighting. We have some matches, which can provide light for 15 seconds once we strike them. It takes us 10 seconds to change a fuse, and we must have continuous, uninterrupted light in order to successfully change it. Given that we only have two hands (one which must hold the match), we can only fix one fuse at a time. How can we go about fixing 3 fuses when we have 3 matches?

### (2.2) Solution

As you might expect, we'll begin by importing the necessary libraries:

In [1]:
import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.io import PDDLWriter

Next, we can begin defining our domain by creating our custom types - in this case we only need types for the matches and for the fuses:

In [2]:
Match = UserType('Match')
Fuse = UserType('Fuse')

Now we will define our predicates, which will all be boolean (we don't need any numeric fluents for this problem):

In [3]:
handfree = Fluent('handfree') # defaults to BoolType()
light = Fluent('light') # defaults to BoolType()
match_used = Fluent('match_used', BoolType(), m=Match)
fuse_mended = Fluent('fuse_mended', BoolType(), f=Fuse)

print(light.type)

bool


Notice that if we don't specify the value type for each fluent, the default is just boolean. Next up, we need to define our actions, which in this case are all durative. We can define durative actions (which are stored in a `DurativeAction` object) as follows:

In [4]:
light_match = DurativeAction('light_match', m=Match)
m = light_match.parameter('m')
light_match.set_fixed_duration(15) # This declares the total duration of the action
light_match.add_condition(StartTiming(), Not(match_used(m))) # This predicate must be true at the start of the action
light_match.add_effect(StartTiming(), match_used(m), True) # This predicate will no longer be true after the start of the action
light_match.add_effect(StartTiming(), light, True) # This predicate will no longer be true after the start of the action
light_match.add_effect(EndTiming(), light, False) # This predicate will become true at the end of the action

fix_fuse = DurativeAction('fix_fuse', f=Fuse)
f = fix_fuse.parameter('f')
fix_fuse.set_fixed_duration(10)
fix_fuse.add_condition(StartTiming(), handfree)
fix_fuse.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), light) # This predicate must be true over the course of the whole action
fix_fuse.add_effect(StartTiming(), handfree, False)
fix_fuse.add_effect(EndTiming(), fuse_mended(f), True)
fix_fuse.add_effect(EndTiming(), handfree, True)

Here we use the `.set_fixed_duration` method to assign the duration (in arbitrary units of time) of the action, and we use the `.add_condition` method to define the temporal conditions (not preconditions!) required to conduct the action. For the `light_match` action (which lights the match), we require that the match object will not have been already lit at the start of the action, and we state that this action will cause the match to have been used immediately after the start of the action, and will provide light once the action begins and no light once the action ends.

For the `fix_fuse` action (which fixes the fuse), we require that our hand will be free at the start of the action and that there will be light from the start of the action until its end. We also state that this action will cause our hand to no longer be free from the start of the action, the fuse to be fixed once the action ends, and our hand to be free at the action's end.

That's it for our domain definition, and now we can begin building the problem. First, we'll instantiate a new `Problem` object:

In [5]:
problem_match = Problem('MatchCellar')

Next we'll instantiate 3 fuses and 3 matches and add them to the problem, along with the predicates and actions:

In [6]:
fuses = [Object(f'F{i}', Fuse) for i in range(1,4)]
matches = [Object(f'M{i}', Match) for i in range(1,4)]
problem_match.add_objects(fuses)
problem_match.add_objects(matches)

problem_match.add_fluent(handfree)
problem_match.add_fluent(light)
problem_match.add_fluent(match_used, default_initial_value=False)
problem_match.add_fluent(fuse_mended , default_initial_value=False)

problem_match.add_action(light_match)
problem_match.add_action(fix_fuse)

Then we can define the initial state (which we really could've done when we added the fluents to the problem, but will be done explicitly here for clarity):

In [7]:
problem_match.set_initial_value(light, False)
problem_match.set_initial_value(handfree, True)

print(problem_match.initial_values)

{light: false, handfree: true, match_used(M1): false, match_used(M2): false, match_used(M3): false, fuse_mended(F1): false, fuse_mended(F2): false, fuse_mended(F3): false}


Finally, we can specify the goals for this problem (i.e. that all of the fuses will be fixed):

In [8]:
for f in fuses:
    problem_match.add_goal(fuse_mended(f))

Our problem is now fully defined, and we can check its type and its contents using the following commands:

In [9]:
print(problem_match.kind)
print()
print(problem_match)

PROBLEM_CLASS: ['ACTION_BASED']
TIME: ['CONTINUOUS_TIME']
CONDITIONS_KIND: ['NEGATIVE_CONDITIONS']
TYPING: ['FLAT_TYPING']

problem name = MatchCellar

types = [Fuse, Match]

fluents = [
  bool handfree
  bool light
  bool match_used[m=Match]
  bool fuse_mended[f=Fuse]
]

actions = [
  durative action light_match(Match m) {
    duration = [15, 15]
    conditions = [
      [start]:
        (not match_used(m))
    ]
    effects = [
      start:
        match_used(m) := true:
        light := true:
      end:
        light := false:
    ]
    simulated effects = [
    ]
  }
  durative action fix_fuse(Fuse f) {
    duration = [10, 10]
    conditions = [
      [start]:
        handfree
      [start, end]:
        light
    ]
    effects = [
      start:
        handfree := false:
      end:
        fuse_mended(f) := true:
        handfree := true:
    ]
    simulated effects = [
    ]
  }
]

objects = [
  Fuse: [F1, F2, F3]
  Match: [M1, M2, M3]
]

initial fluents default = [
  bool match_u

Notice that the only new problem feature that we haven't seen yet is called `CONTINUOUS_TIME` - it is one of the features of temporal planning problems in the UPF (but not the only one - we'll see more later), and it corresponds to the `:durative-actions` requirement for our domain. Let's write this problem to the appropriate PDDL files (this time, they're PDDL 2.1!), using the commands we've seen before:

In [10]:
w = PDDLWriter(problem_match)
w.write_domain('MatchCellar/domain_match_upf.pddl')
w.write_problem('MatchCellar/problem_match_upf.pddl')

Finally, we can actually solve our temporal planning problem with ease using the UPF. We need to install a planner called [Tamer](https://tamer.fbk.eu) in order to solve these kinds of problems, and we can do so by running the following command in our terminal (don't forget to activate the `cogrob` virtual environemnt first, if you're using it):

`pip install up-tamer==0.2.0.23.dev1`

In [11]:
with OneshotPlanner(problem_kind=problem_match.kind) as planner:
    result = planner.solve(problem_match)
    plan = result.plan
    if plan is not None:
        print("%s returned:" % planner.name)
        for start, action, duration in plan.timed_actions:
            print("%s: %s [%s]" % (float(start), action, float(duration)))
    else:
        print("No plan found.")

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 412 of `c:\Users\yotam\.conda\envs\cogrob\lib\site-packages\unified_planning\shortcuts.py`, you are using the following planning engine:
  * Engine name: Tamer
  * Developers:  FBK Tamer Development Team
  * Description: Tamer offers the capability to generate a plan for classical, numerical and temporal problems.
  *              For those kind of problems tamer also offers the possibility of validating a submitted plan.

Tamer returned:
0.0: light_match(M2) [15.0]
0.01: fix_fuse(F2) [10.0]
15.01: light_match(M1) [15.0]
15.02: fix_fuse(F3) [10.0]
30.02: light_match(M3) [15.0]
30.03: fix_fuse(F1) [10.0]


The plan returned by Tamer tells us to first light a match at $t=0.00$, and then immediately start fixing the first fuse at $t=0.01$ (remember that the light is turned on only immediately *after* the `light_match` action begins - thus there's a 0.01 unit delay until we start fixing the fuse). We'll finish fixing the first fuse at $t=10.01$, but we don't have time to start fixing the second fuse since the first match will burn out at $t=15.00$ (and we must have uninterrupted light throughout the entire fuse-fixing action in order to actually do it). Thus, we will have to wait until $t=15.01$ to light our second match, which then allows us to start fixing the second fuse at $t=15.02$. This fuse will finish being fixed at $t=25.02$, but we will have to wait until the second match burns out at $t=30.01$ in order to light the third and final match at $t=30.02$. Once that third match is lit, we can start fixing the final fuse (at $t=30.03$), and we will be done fixing all of the fuses at $t=40.03$ (and the last match will burn out at $t=45.02$).

In total, it takes us 40.03 units of time in order to complete all of our goals and fix every fuse. Notice that here we had actions which were *concurrent* - they occur simultaneously in the plan, which would be impossible to represent using classical planning.

Finally, I've added the code for this problem as a `.py` file under the folder `MatchCellar`, and you can run it directly from there using the following command:


In [12]:
from MatchCellar import match_upf
match_upf.main()

Tamer returned:
0.0: light_match(M2) [15.0]
0.01: fix_fuse(F2) [10.0]
15.01: light_match(M1) [15.0]
15.02: fix_fuse(F3) [10.0]
30.02: light_match(M3) [15.0]
30.03: fix_fuse(F1) [10.0]


## (3) Ex: Satellite Activity Planning (Numeric)

Next let's try our hand at a problem that's a bit more complicated, but we'll limit it first to be a numeric planning problem (later we will turn it into a temporal planning problem). It is based on the Satellite domain from the AIPS 2002 planning competition.

### (3.1) Problem Statement

The Israeli Space Agency (ISA) has recently launched a new satellite program for which they will send a fleet of satellites to geosynchronous orbit (GEO) in order to conduct measurements of various objects in our galaxy. They would like to observe a set of stars and celestial phenomena using measurement devices from the following set: imagers (cameras), thermographs, and spectrographs. At first, they send one satellite with an imager to GEO in order to observe a certain star and a certain phenomenon in our galaxy.

In order to record data (i.e. take pictures), the satellite must first turn on the imaging instrument, and then it must calibrate it. In order to calibrate it, the satellite must be pointing directly towards the ground control station (GCS), which is located on Earth. If the satellite is not already pointing at the GCS, it must use fuel in order to turn towards it - the satellite begins its mission with a set amount of fuel (in this case 240 grams), and each time it turns to point at a different object it spends a certain amount of fuel.

Once the instrument is calibrated, the satellite can start using it to record data. To take images of a body, the satellite must be pointing at that body and it must have enough data storage left in order to keep the incoming data from the measurement process. The satellite initially starts with 500 GB, and it will need 150 GB to record data from the phenomenon and 250 GB to record data from the star.

The fuel required to turn between two directions (as well as the initial pointing direction) is given in the following figure:
<p align="center">
  <img src="Graphics/SatNum.png" width=1200/>
</p>

Will the satellite be able to capture images of both the star and the phenomenon before running out of fuel or data storage? If so, how much of each will be left at the end of the mission?

### (3.2) Solution

We can begin by defining the necessary types:

In [14]:
Satellite = UserType("Satellite")
Direction = UserType("Direction")
Instrument = UserType("Instrument") # one instrument can conduct different types of measurements
Mode = UserType("Mode") # the type of measurement conducted

Next, we can define our fluents, starting with the predicates:

In [15]:
On_Board = Fluent("On_Board", BoolType(), i=Instrument, s=Satellite)
Supports = Fluent("Supports", BoolType(), i=Instrument, m=Mode) # Which measurements can the instrument conduct?
Pointing = Fluent("Pointing", BoolType(), s=Satellite, d=Direction) 
Power_Avail = Fluent("Power_Avail", BoolType(), s=Satellite) # If the satellite is not currently measuring anything, there will be power available to start recording data
Power_On = Fluent("Power_On", BoolType(), i=Instrument) # Is the instrument receiving power from the satellite? If yes, it can record data
Calibrated = Fluent("Calibrated", BoolType(), i=Instrument) # Is the instrument calibrated?
Have_Image = Fluent("Have_Image", BoolType(), d=Direction, m=Mode) # Did the satellite record data on the given body (called a direction here) already?
Calibration_Target = Fluent("Calibration_Target", BoolType(), i=Instrument, d=Direction) # Indicates which direction (usually a ground station) can calibrate the specified instrument

followed by the numeric fluents:

In [16]:
Data_Capacity = Fluent("Data_Capacity", IntType(), s=Satellite) # How much data storage does the satellite have left?
Data = Fluent("Data", IntType(), d=Direction, m=Mode) # How much data is recorded on a certain body (direction) in the specified mode?
Slew_Time = Fluent("Slew_Time", IntType(), a=Direction, b=Direction) # How long (i.e. how much fuel) does it take for the satellite to turn between two pointing directions?
Data_Stored = Fluent("Data_Stored", IntType()) # How much data has been recorded in total (across all satellites)?
Fuel = Fluent("Fuel", IntType(), s=Satellite) # How much fuel does the satellite have left?
Fuel_Used = Fluent("Fuel_Used", IntType()) # How much fuel has been spent in total (across all satellites)?

Next up are the actions, defined as follows:

In [17]:
# Turns the satellite between two pointing directions
turn_to = InstantaneousAction('turn_to', s=Satellite, d_new=Direction, d_prev=Direction)
s = turn_to.parameter('s')
d_new = turn_to.parameter('d_new')
d_prev = turn_to.parameter('d_prev')
turn_to.add_precondition(Pointing(s, d_prev))
turn_to.add_precondition(Not(Equals(d_prev, d_new)))
turn_to.add_precondition(GE(Fuel(s), Slew_Time(d_new, d_prev))) # Ensures we have enough fuel to conduct this maneuver
turn_to.add_effect(Pointing(s, d_new), True)
turn_to.add_effect(Pointing(s, d_prev), False)
turn_to.add_effect(Fuel(s), Fuel(s) - Slew_Time(d_new, d_prev)) # Update value for the satellite's fuel
turn_to.add_effect(Fuel_Used, Fuel_Used + Slew_Time(d_new, d_prev)) # Update value for the total fuel spent

# Turns on the instrument so that it will get power from the satellite
switch_on = InstantaneousAction('switch_on', i=Instrument, s=Satellite)
i = switch_on.parameter('i')
s = switch_on.parameter('s')
switch_on.add_precondition(On_Board(i,s))
switch_on.add_precondition(Power_Avail(s))
switch_on.add_effect(Power_On(i), True)
switch_on.add_effect(Calibrated(i), False)
switch_on.add_effect(Power_Avail(s), False)

# Turns off the instrument so that it no longer draws power from the satellite
switch_off = InstantaneousAction('switch_off', i=Instrument, s=Satellite)
i = switch_off.parameter('i')
s = switch_off.parameter('s')
switch_off.add_precondition(On_Board(i,s))
switch_off.add_precondition(Power_On(i))
switch_off.add_effect(Power_On(i), False)
switch_off.add_effect(Power_Avail(s), True)

# Calibrates the instrument so that it can record data
calibrate = InstantaneousAction('calibrate', s=Satellite, i=Instrument, d=Direction)
s = calibrate.parameter('s')
i = calibrate.parameter('i')
d = calibrate.parameter('d')
calibrate.add_precondition(On_Board(i,s))
calibrate.add_precondition(Calibration_Target(i,d)) # Ensures that the satellite is pointing towards a valid calibration target (i.e. a ground station)
calibrate.add_precondition(Pointing(s, d))
calibrate.add_precondition(Power_On(i))
calibrate.add_effect(Calibrated(i), True)

# Records the data
take_image = InstantaneousAction('take_image', s=Satellite, d=Direction, i=Instrument, m=Mode)
s = take_image.parameter('s')
d = take_image.parameter('d')
i = take_image.parameter('i')
m = take_image.parameter('m')
take_image.add_precondition(Calibrated(i))
take_image.add_precondition(On_Board(i,s))
take_image.add_precondition(Supports(i,m))
take_image.add_precondition(Power_On(i))
take_image.add_precondition(Pointing(s,d))
take_image.add_precondition(GE(Data_Capacity(s), Data(d,m))) # Ensures we have enough data storage to record this data
take_image.add_effect(Have_Image(d,m), True)    
take_image.add_effect(Data_Capacity(s), Data_Capacity(s) - Data(d,m)) # Update value for the satellite's data capacity
take_image.add_effect(Data_Stored, Data_Stored + Data(d,m)) # Update value for the total data recorded

Our domain definition is done, and now we can begin building our problem. First we declare the problem name, instantiate the objects, and add the objects to the problem:

In [18]:
problem_satellite_num = Problem("satellite_num")

satellite0 = Object("satellite0", Satellite)
instrument0 = Object("instrument0", Instrument)
image0 = Object("image0", Mode)
Star0 = Object("Star0", Direction)
GroundStation1 = Object("GroundStation1", Direction)
Phenomenon2 = Object("Phenomenon2", Direction)
objects = [satellite0, instrument0, image0, Star0, GroundStation1, Phenomenon2]

problem_satellite_num.add_objects(objects)

Now we add the predicates to the problem:

In [19]:
problem_satellite_num.add_fluent(On_Board, default_initial_value=False)
problem_satellite_num.add_fluent(Supports, default_initial_value=False)
problem_satellite_num.add_fluent(Pointing, default_initial_value=False)
problem_satellite_num.add_fluent(Power_Avail, default_initial_value=True) # power is available for the instruments from the beginning, for all satellites
problem_satellite_num.add_fluent(Power_On, default_initial_value=False)
problem_satellite_num.add_fluent(Calibrated, default_initial_value=False)
problem_satellite_num.add_fluent(Have_Image, default_initial_value=False)
problem_satellite_num.add_fluent(Calibration_Target, default_initial_value=False)

bool Calibration_Target[i=Instrument, d=Direction]

along with the numeric fluents:

In [20]:
problem_satellite_num.add_fluent(Data_Capacity)
problem_satellite_num.add_fluent(Data, default_initial_value=0)
problem_satellite_num.add_fluent(Slew_Time, default_initial_value=0)
problem_satellite_num.add_fluent(Data_Stored, default_initial_value=0) # the total data recorded at the start is 0
problem_satellite_num.add_fluent(Fuel)
problem_satellite_num.add_fluent(Fuel_Used, default_initial_value=0) # the total fuel used at the start is 0

integer Fuel_Used

and finally the actions:

In [21]:
problem_satellite_num.add_action(turn_to)
problem_satellite_num.add_action(switch_on)
problem_satellite_num.add_action(switch_off)
problem_satellite_num.add_action(calibrate)
problem_satellite_num.add_action(take_image)

Next up, we must define the initial state for our problem. We start with the initial values of the predicates:

In [22]:
problem_satellite_num.set_initial_value(Supports(instrument0,image0), True)
problem_satellite_num.set_initial_value(Calibration_Target(instrument0,GroundStation1), True)
problem_satellite_num.set_initial_value(On_Board(instrument0,satellite0), True)
problem_satellite_num.set_initial_value(Power_Avail(satellite0), True) # Did we need this?
problem_satellite_num.set_initial_value(Pointing(satellite0, Phenomenon2), True)

and then for the initial values of the numeric fluents:

In [23]:
problem_satellite_num.set_initial_value(Data_Capacity(satellite0), 550)
problem_satellite_num.set_initial_value(Fuel(satellite0), 240)
problem_satellite_num.set_initial_value(Data(Phenomenon2, image0), 150)
problem_satellite_num.set_initial_value(Data(Star0, image0), 250)

# In order to set the slew times between pointing directions, we will create a dictionary and iterate over it in order to set the initial values for turning between every pair of directions
slew_dict = {Star0 : {GroundStation1 : 18, Phenomenon2 : 14},
             GroundStation1 : {Star0 : 18, Phenomenon2 : 89},
             Phenomenon2 : {Star0 : 14, GroundStation1 : 89}}
for d1 in slew_dict:
    for d2 in slew_dict[d1]:
        problem_satellite_num.set_initial_value(Slew_Time(d1,d2), slew_dict[d1][d2])

Finally, we can set the goals for this problem:

In [24]:
problem_satellite_num.add_goal(Have_Image(Star0, image0))
problem_satellite_num.add_goal(Have_Image(Phenomenon2, image0))

Unfortunately, the UPF doesn't have great support for metrics yet (i.e. we don't have any planners integrated into the UPF that can deal with problems where we want to minimize/maximize a certain numeric fluent), so for now we will have to suffice with a non-optimal solution (if we were to define optimality using the total fuel spent, for example). If we wanted to add such a metric, this would be the line of code necessary to do so:
```
from unified_planning.model.metrics import *
problem_satellite_num.add_quality_metric(MinimizeExpressionOnFinalState(Fuel_Used()))
```
Anyways, now that our problem is fully defined, we can save its PDDL files:

In [25]:
w = PDDLWriter(problem_satellite_num)
w.write_domain('SatelliteNum/domain_satellite_num_upf.pddl')
w.write_problem('SatelliteNum/problem_satellite_num_upf.pddl')

and then solve it using the Tamer planner:

In [26]:
def Get_Final_Values(problem, plan):
    fuel = problem.fluent("Fuel")
    slew = problem.fluent("Slew_Time")
    data_cap = problem.fluent("Data_Capacity")
    data = problem.fluent("Data")
    sat = problem.object("satellite0")
   
    fuel_left = problem.initial_value(fuel(sat))._content.payload
    data_left = problem.initial_value(data_cap(sat))._content.payload
    print(f"Initial Fuel Level: {fuel_left}, Initial Data Capacity: {data_left}")

    for a in plan.actions:
        if a.action.name == 'turn_to':
            d_new = a.actual_parameters[1]
            d_prev = a.actual_parameters[2]
            fuel_spent = problem.initial_value(slew(d_new, d_prev))._content.payload
            fuel_left -= fuel_spent
        elif a.action.name == 'take_image':
            d = a.actual_parameters[1]
            m = a.actual_parameters[3]
            data_spent = problem.initial_value(data(d, m))._content.payload
            data_left -= data_spent
    print(f"Final Fuel Level: {fuel_left}, Final Data Capacity: {data_left}")
    return

up.shortcuts.get_env().credits_stream = None
with OneshotPlanner(name='tamer') as planner:
    result = planner.solve(problem_satellite_num)
    plan = result.plan
    if plan is not None:
        print("%s returned:" % planner.name)
        print(plan)
        print()
        Get_Final_Values(problem_satellite_num, plan)
    else:
        print("No plan found.")

Tamer returned:
[switch_on(instrument0, satellite0), turn_to(satellite0, GroundStation1, Phenomenon2), calibrate(satellite0, instrument0, GroundStation1), turn_to(satellite0, Phenomenon2, GroundStation1), take_image(satellite0, Phenomenon2, instrument0, image0), turn_to(satellite0, Star0, Phenomenon2), take_image(satellite0, Star0, instrument0, image0)]

Initial Fuel Level: 240, Initial Data Capacity: 550
Final Fuel Level: 48, Final Data Capacity: 150


Thus, our solution requires that we first turn on the imaging instrument, then turn the satellite from pointing towards the phenomenon to pointing towards the ground station, and then calibrate the instrument. Once that's done, we can turn the satellite back towards the phenomenon and record data, and once that's done we can turn the satellite towards the star and again record data. Upon doing so, we have completed our tasks, and we have spent 192 grams of fuel and 400 GB of data storage to do so. I've added the code for this problem as a `.py` file under the folder `SatelliteNum`, and you can run it directly from there using the following command:

In [27]:
from SatelliteNum import satellite_num_upf
satellite_num_upf.main()

Tamer returned:
[turn_to(satellite0, GroundStation1, Phenomenon2), switch_on(instrument0, satellite0), calibrate(satellite0, instrument0, GroundStation1), turn_to(satellite0, Phenomenon2, GroundStation1), take_image(satellite0, Phenomenon2, instrument0, image0), turn_to(satellite0, Star0, Phenomenon2), take_image(satellite0, Star0, instrument0, image0)]

Initial Fuel Level: 240, Initial Data Capacity: 550
Final Fuel Level: 48, Final Data Capacity: 150


It turns out that the PDDL extension for VS Code is also capable of solving this problem, and if you run its solver on the `problem_satellite_num.pddl` file in the `SatelliteNum` folder then you will actually obtain the optimal solution in terms of total fuel spent:
<p align="center">
  <img src="Graphics/NumSol.png" width=800/>
</p>
The following plots show the evolution of the numeric fluent values over the course of the produced plan, including the metric we defined (the total fuel spent):
<p align="center">
  <img src="Graphics/NumGraphs1.png" width=500/>
  <img src="Graphics/NumGraphs2.png" width=500/>
</p>

## (4) Ex: Satellite Activity Planning (Temporal)

Finally, let's try to solve a different variation of the satellite problem which relies on temporal (rather than numeric) planning. This problem is again based on the Satellite domain from the AIPS 2002 planning competition.

### (4.1) Problem Statement

The ISA's initial pilot for the new satellite program was a success! They are now looking to add a new satellite to the fleet (with both a thermograph and a spectrograph) in order to more efficiently image 2 new stars and 3 new celestial phenomena. They want to record thermograph data for both stars, and spectrograph data for all 3 phenomena. This time, they no longer have any constraints on the fuel expendtiture nor on the data storage capacity, but rather they would like to record all of the necessary data within a certain amount of time.

The initial setup of the problem is shown in the following figure:
<p align="center">
  <img src="Graphics/SatTemp0.png" width=1200/>
</p>

How much time will it take the satellite to record all of the data and complete the mission?

### (4.2) Solution

We will discuss the solution together in class. The solution `.py` file for this problem can be found in the `SatelliteTemp` folder, and it can be run here using the following commands:

In [29]:
from SatelliteTemp import satellite_temp_upf
satellite_temp_upf.main()

Tamer returned:
0.0: switch_on(instrument1, satellite1) [2.0]
2.01: turn_to(satellite1, GroundStation1, Phenomenon2) [5.0]
7.02: calibrate(satellite1, instrument1, GroundStation1) [5.0]
12.03: turn_to(satellite1, Phenomenon2, GroundStation1) [5.0]
17.04: take_image(satellite1, Phenomenon2, instrument1, spectrograph1) [7.0]
24.04: turn_to(satellite1, Phenomenon1, Phenomenon2) [5.0]
29.05: take_image(satellite1, Phenomenon1, instrument1, spectrograph1) [7.0]
36.06: turn_to(satellite1, Phenomenon3, Phenomenon1) [5.0]
41.07: take_image(satellite1, Phenomenon3, instrument1, spectrograph1) [7.0]
48.08: turn_to(satellite1, Star1, Phenomenon3) [5.0]
53.09: take_image(satellite1, Star1, instrument1, thermograph1) [7.0]
60.1: turn_to(satellite1, Star2, Star1) [5.0]
65.11: take_image(satellite1, Star2, instrument1, thermograph1) [7.0]


## (5) Conclusion

In this tutorial, we:
* Introduced PDDL 2.1 for representations of temporal and numeric planning problems
* Utilized the UPF in order to model and solve temporal/numeric planning problems using Python

Next week we will begin learning about the Linux operating system, so that we can be better prepared to start working with the Robot Operating System (ROS) later on!

#### ***Credit:** This tutorial was written by Yotam Granov, Winter 2022.*

### **References**

[1] AIPlan4EU. ["The Unified Planning Library"](https://unified-planning.readthedocs.io/en/latest/index.html), 2021. GitHub: https://github.com/aiplan4eu/unified-planning

[2] Artificial Intelligence Planning and Scheduling Conference 2002. [The Third International Planning Competition](https://ipc02.icaps-conference.org/), 2002.

[3] Maria Fox & Derek Long. ["PDDL 2.1: An Extension to PDDL for Expressing Temporal Planning Domains"](https://arxiv.org/pdf/1106.4561.pdf), 2003.

[4] Adam Green. ["PDDL 2.1"](https://planning.wiki/ref/pddl21), *Planning.wiki - The AI Planning & PDDL Wiki*.

[5] Alessandro Valentini, Andrea Micheli, & Alessandro Cimatti. ["Temporal Planning with Intermediate Conditions and Effects."](https://ojs.aaai.org/index.php/AAAI/article/view/6553), 2020. Homepage for Tamer: https://tamer.fbk.eu/